In [2]:
import numpy
import screed

In [3]:
def load_refsizes(filename):
    d = {}
    for record in screed.open(filename):
        d[record.name.split()[0]] = len(record.sequence)
    return d

In [4]:
def load_coords(filename):
    lines = [ x.strip() for x in (open(filename)) ]
    assert lines[1].startswith('NUCMER'), lines[0]
    
    coords = []
    for line_no in range(5, len(lines)):
        line = lines[line_no].split()
        s1, e1 = int(line[0]), int(line[1])
        s2, e2 = int(line[3]), int(line[4])
        ident = float(line[9])
        name1, name2 = line[11], line[12]
        s1, e1 = min(s1, e1), max(s1, e1)
        s2, e2 = min(s2, e2), max(s2, e2)
        coords.append((s1, e1, s2, e2, ident, name1, name2))
    return coords



In [5]:
IDENT_THRESHOLD = 99
LENGTH_THRESHOLD = 100

def make_covered_ivals(refsizes, coords):
    # let's make some intervals
    ivals = {}
    for genome in refsizes:
        ivals[genome] = numpy.zeros(refsizes[genome])

    # now fill in the intervals with what is covered by a nucmer alignment
    for s1, e1, s2, e2, ident, name1, name2 in coords:
        if ident < IDENT_THRESHOLD or e2 - s2 < LENGTH_THRESHOLD:
            continue
        genome = ivals[name1]
        genome[s1 - 1:e1] = numpy.ones(e1 - s1 + 1)
    return ivals

In [6]:
def calc_coverage(ivals):
    covered = 0.
    total = 0.
    for k, v in ivals.items():
        t = len(v)
        c = sum(v)
        total += t
        covered += c
        #if c / t < .9: # print names of genomes with less than 90% coverage
        #    print k, c/t

    return covered/total

In [7]:
WINDOW=100

def _return_uncovered(cov):
    length = len(cov)
    cov2 = numpy.zeros(length)
    for i in numpy.where(cov == 0)[0]: # find all uncovered bases
        j = i
        # iterate from first uncovered base until you find covered,
        while j < length and cov[j] == 0 \
                and cov2[j] == 0:  # OR you discover that we've already seen this one
            j += 1

        if j - i >= WINDOW:                  # where length of uncovered region big enough, set cov2
            cov2[i:j] = numpy.ones(j - i)    # this is the interval of the uncovered region

    return cov2

def make_uncovered_ivals(ivals):
    e = {}
    for k, v in ivals.items():
        e[k] = _return_uncovered(v)
    return e

In [8]:
megahit_coords = load_coords('megahit.coords')
idba_coords = load_coords('idba.coords')
refsizes = load_refsizes('mircea.fa')

In [9]:
print 'megahit'
megahit_ivals = make_covered_ivals(refsizes, megahit_coords)
print 'idba'
idba_ivals = make_covered_ivals(refsizes, idba_coords)
print 'done'

megahit
idba
done


In [10]:
print 'megahit2'
megahit_ivals_uncov = make_uncovered_ivals(megahit_ivals)
print 'idba2'
idba_ivals_uncov = make_uncovered_ivals(idba_ivals)
print 'done'

megahit2
idba2
done


In [11]:
print(len(idba_ivals_uncov))

64


In [12]:
print len(megahit_coords)
print len(refsizes)

print calc_coverage(megahit_ivals)
print calc_coverage(idba_ivals)

100020
64
0.90605623055
0.895634677613


In [13]:
missing_refs = set()
for s1, e1, s2, e2, ident, name1, name2 in megahit_coords:
    if name1 not in refsizes:
        assert 0


In [14]:
k = 'Shewanella_baltica_OS185'
sum(idba_ivals[k]) / len(idba_ivals[k])


0.74722344706737653

In [15]:
neither = 0.
both = 0.
either = 0.

for k in megahit_ivals:
    final = megahit_ivals_uncov[k] + idba_ivals_uncov[k]
    neither += len(numpy.where(final == 0)[0])
    either += len(numpy.where(final == 1)[0])
    both += len(numpy.where(final == 2)[0])

In [16]:
print neither, either, both

180196380.0 10665463.0 14741872.0


In [17]:
total = neither + either + both
print neither /total
print either/total
print both/total

0.876425700771
0.0518738827263
0.0717004165027
